# Generatore Monte Carlo di raggi cosmici

La distribuzione angolare dei raggi cosmici che raggiungono il suolo terresrre segue approssimativamente la legge $\mathrm{d}P(\theta,\phi)/\mathrm{d}\Omega = k \cos^2\theta$ con $0\leq\theta<\pi/2$.

Il punto di impatto al suolo è uniformemente distribuito sulla superficie.

* Scrivere un generatore di raggi cosmici distribuiti secondo questo modello.
* Graficarne la distribuzione del punto di impatto e della direzione di provenienza.
* Graficare un numero finito con l'utility di matplotlib [quiver](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.quiver.html), che è pensata per visualizzare un campo vettoriale, ma noi la useremo in modo diverso

Opzionale:

* Definire la geometria di un semplice telescopio, ad es.: fatto da due piani quadrati orizzontali posti ad una certa altezza tra di loro
* Calcolare le intersezioni con i due piani di rivelatore dei raggi cosmici generati aventi punto di impatto sul piano di base
* Calcolare l'accettanza geometrica del telescopio rispetto ai raggi cosmici che arrivano al rivelatore di base

## Distribuzione angolare

La prima cosa da fere è generare punti in due dimensioni, $(\theta, \phi)$, tali che i punti siani distribuiti secondo la legge:

$\displaystyle \frac{\mathrm{d}P}{\mathrm{d}\Omega} = \frac{\mathrm{d}P}{\sin\theta\mathrm{d}\theta\,\mathrm{d}\phi} = k \cos^2\theta$ 

dove $k$ è una costante di normalizzazione e $0\le \theta <\pi/2$
Possiamo anche riscrivere:

$\displaystyle\frac{\mathrm{d}P}{\mathrm{d}\theta\,\mathrm{d}\phi}=f(\theta)\,g(\phi)= k\sin\theta\cos^2\theta$ 

Dove:

$f(\theta)=c_1\sin \theta\cos^2\theta$

$g(\phi) = c_2$

e $k=c_1 c_2$.

La distribuzione cumulativa di $f(\theta)$ è:

$\displaystyle F(\theta) = c_1 \int_{0}^\theta \sin \theta^\prime\cos^2\theta^\prime\,\mathrm{d}\theta^\prime = 
-\frac{c_1}{3}\left[\cos^3\theta^\prime\right]_{\theta^\prime=0}^{\theta^\prime=\theta} = 
\frac{c_1}{3}(1 - \cos^3\theta)$

Il coefficiente di normalizzazionre $c_1$ va posto in modo che sia $F(\pi/2) = 1$:

$\displaystyle 1 = \frac{c_1}{3}\left(1 - \cos^3 \frac{\pi}{2}\right) = \frac{c_1}{3}$

Quindi $c_1 = 3$ e $F(\theta) = 1 - \cos^3\theta$.

$F(\theta)$ va invertita per otterenere $\theta$ dall'equazione: $F(\theta)= r_1$, dove $r_1$ è un numero uniormemente distribuito in $[0,1\,[$:


$\displaystyle 1 - \cos^3\theta = r_1$

Da cui:


$\displaystyle\theta = \mathrm{arccos}(\sqrt[3]{1-r_1})\in \left[0, \frac{\pi}{2}\right[$

Considerando che la distribuzione di $1-r_1$ è la stessa di $r_1$, potremmo anche scrivere:

$\displaystyle\theta = \mathrm{arccos}(\sqrt[3]{r_1})\in \left] 0, \frac{\pi}{2}\right]$


L'angolo $\phi$, come di consueto, si può generare unifrmemente tra $0$ e $2\pi$ a partire da un numero $r_2$ distribuito uniformemente in $[0,1\,[$:
$\phi = 2\pi r_2\in [0, 2\pi\,[$